In [1]:
import pandas as pd
import geopandas as gpd
import altair as alt

In [2]:
all_sheets = pd.read_excel('https://docs.google.com/spreadsheets/d/1yZv9w9zRKwrGTaR-YzmAqMefw4wMlaXocejdxZaTs6w/export?format=xlsx', sheet_name=None)

In [3]:
print(all_sheets.keys())
df = all_sheets['Jan31_2pm']
df.head()

odict_keys(['Jan31_7pm', 'Jan31_2pm', 'Jan30_930pm', 'Jan30_11am', 'Jan29_9pm', 'Jan29_230pm', 'Jan29_130pm', 'Jan28_11pm', 'Jan28_6pm', 'Jan28_1pm', 'Jan27_830pm', 'Jan27_7pm', 'Jan27_9am', 'Jan26_11pm', 'Jan26_11am', 'Jan25_10pm', 'Jan25_12pm', 'Jan25_12am', 'Jan24_12pm', 'Jan24_12am', 'Jan23_12pm', 'Jan22_12pm', 'Jan22_12am'])


,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,Hubei,Mainland China,2020-01-31 14:00:00,5806,204.0,141.0
1,Zhejiang,Mainland China,2020-01-31 14:00:00,538,NaN,14.0
2,Guangdong,Mainland China,2020-01-31 14:00:00,436,NaN,11.0
3,Henan,Mainland China,2020-01-31 14:00:00,352,2.0,3.0
4,Hunan,Mainland China,2020-01-31 14:00:00,332,NaN,2.0


In [25]:
selection_sheets = list(all_sheets.keys())[::3]
print(selection_sheets)

['Jan31_7pm', 'Jan30_11am', 'Jan29_130pm', 'Jan28_1pm', 'Jan27_9am', 'Jan25_10pm', 'Jan24_12pm', 'Jan22_12pm']


In [40]:
selection_sheets = ['Jan31_7pm', 'Jan30_930pm', 'Jan29_230pm', 'Jan28_11pm', 'Jan27_7pm', 'Jan26_11pm', 'Jan25_10pm', 'Jan24_12pm', 'Jan23_12pm', 'Jan22_12pm']

In [41]:
df_all = pd.DataFrame()
for sheet in selection_sheets:
    df = all_sheets[sheet]
    df_all = pd.concat((df_all, df), axis=0, sort=False)

In [42]:
df_all.replace(
    to_replace=['Ningxia', 'Inner Mongolia', 'Xinjiang', 'Tibet'], 
    value=['Nei Mongol', 'Ningxia Hui','Xinjiang Uygur', 'Xizang'],
    inplace=True
)
df_all.fillna(0, inplace=True)

In [7]:
url_china_prv = 'https://raw.githubusercontent.com/deldersveld/topojson/master/countries/china/china-provinces.json'
gdf = gpd.read_file(url_china_prv, driver='TopoJSON')
gdf.head()

,id,ISO,NAME_0,ID_1,NAME_1,TYPE_1,ENGTYPE_1,NL_NAME_1,VARNAME_1,geometry
0,None,CHN,China,1,Anhui,Shěng,Province,安徽|安徽,Ānhuī,"POLYGON ((119.63215 31.13933, 119.64444 31.115..."
1,None,CHN,China,2,Beijing,Zhíxiáshì,Municipality,北京|北京,Běijīng,"POLYGON ((117.37966 40.22687, 117.38301 40.208..."
2,None,CHN,China,3,Chongqing,Zhíxiáshì,Municipality,重慶|重庆,Chóngqìng,"POLYGON ((109.57656 31.73312, 109.60255 31.719..."
3,None,CHN,China,4,Fujian,Shěng,Province,福建,Fújiàn,"MULTIPOLYGON (((117.70771 23.78171, 117.72001 ..."
4,None,CHN,China,5,Gansu,Shěng,Province,甘肅|甘肃,Gānsù,"POLYGON ((97.18466 42.77164, 97.39479 42.41457..."


In [8]:
import topojson as tp
t = tp.Topology(gdf)

In [9]:
gdf_topo = t.toposimplify(0.09).topoquantize(1e3).to_gdf()

In [44]:
gdf_sel = gdf_topo[['NAME_1', 'geometry']]

In [45]:
gdf_join = pd.merge(gdf_sel, df_all, left_on='NAME_1', right_on='Province/State')

In [46]:
gdf_join.head()

,NAME_1,geometry,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Suspected,Demised,Country
0,Anhui,"POLYGON ((119.63838 31.13065, 119.45455 30.670...",Anhui,Mainland China,2020-01-31 19:00:00,237.0,0.0,3.0,0.0,0.0,0
1,Anhui,"POLYGON ((119.63838 31.13065, 119.45455 30.670...",Anhui,Mainland China,2020-01-30 21:30:00,237.0,0.0,3.0,0.0,0.0,0
2,Anhui,"POLYGON ((119.63838 31.13065, 119.45455 30.670...",Anhui,Mainland China,2020-01-29 14:30:00,152.0,0.0,2.0,0.0,0.0,0
3,Anhui,"POLYGON ((119.63838 31.13065, 119.45455 30.670...",Anhui,Mainland China,2020-01-28 23:00:00,152.0,0.0,0.0,0.0,0.0,0
4,Anhui,"POLYGON ((119.63838 31.13065, 119.45455 30.670...",Anhui,Mainland China,2020-01-27 19:00:00,70.0,0.0,0.0,0.0,0.0,0


In [47]:
gdf_join['Last Update'] = gdf_join['Last Update'].dt.strftime('%Y-%m-%d')
gdf_join.head()

,NAME_1,geometry,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Suspected,Demised,Country
0,Anhui,"POLYGON ((119.63838 31.13065, 119.45455 30.670...",Anhui,Mainland China,2020-01-31,237.0,0.0,3.0,0.0,0.0,0
1,Anhui,"POLYGON ((119.63838 31.13065, 119.45455 30.670...",Anhui,Mainland China,2020-01-30,237.0,0.0,3.0,0.0,0.0,0
2,Anhui,"POLYGON ((119.63838 31.13065, 119.45455 30.670...",Anhui,Mainland China,2020-01-29,152.0,0.0,2.0,0.0,0.0,0
3,Anhui,"POLYGON ((119.63838 31.13065, 119.45455 30.670...",Anhui,Mainland China,2020-01-28,152.0,0.0,0.0,0.0,0.0,0
4,Anhui,"POLYGON ((119.63838 31.13065, 119.45455 30.670...",Anhui,Mainland China,2020-01-27,70.0,0.0,0.0,0.0,0.0,0


In [48]:
gdf_join.rename({'Last Update': 'Update'})
gdf_join_slice = gdf_join[['NAME_1', 'geometry', 'Last Update', 'Confirmed', 'Deaths', 'Recovered']]
gdf_join_slice.columns = ['Province', 'geometry', 'Date', 'Confirmed', 'Deaths', 'Recovered']

In [49]:
gdf_join_slice.head()

,Province,geometry,Date,Confirmed,Deaths,Recovered
0,Anhui,"POLYGON ((119.63838 31.13065, 119.45455 30.670...",2020-01-31,237.0,0.0,3.0
1,Anhui,"POLYGON ((119.63838 31.13065, 119.45455 30.670...",2020-01-30,237.0,0.0,3.0
2,Anhui,"POLYGON ((119.63838 31.13065, 119.45455 30.670...",2020-01-29,152.0,0.0,2.0
3,Anhui,"POLYGON ((119.63838 31.13065, 119.45455 30.670...",2020-01-28,152.0,0.0,0.0
4,Anhui,"POLYGON ((119.63838 31.13065, 119.45455 30.670...",2020-01-27,70.0,0.0,0.0


In [50]:
column_list = ['Confirmed', 'Deaths', 'Recovered']
row_list = gdf_join_slice['Date'].unique().tolist()

In [51]:
charts_conf = []
charts_death = []
charts_recv = []
for date in row_list:
    l = alt.Chart(gdf_join_slice).mark_geoshape(stroke='gray').encode(  
        fill=alt.Fill('Confirmed', type='quantitative', scale=alt.Scale(type='threshold', domain=[10, 50, 100,500, 1000, 5000])),    
        opacity=alt.Opacity('Confirmed', type='quantitative', scale=alt.Scale(type='threshold', domain=[10,50, 100,500, 1000, 5000])),    
        tooltip=['Confirmed', 'Deaths', 'Recovered','Date'],
    ).transform_filter(
        alt.FieldEqualPredicate(field='Date', equal=date)
    ).properties(title=date, width=200, height=150)
    charts_conf.append(l)
    
    m = alt.Chart(gdf_join_slice).mark_geoshape(stroke='gray').encode(  
        fill=alt.Fill('Deaths', type='quantitative', scale=alt.Scale(type='threshold', domain=[1,5,10,50,100,500])),    
        opacity=alt.Opacity('Deaths', type='quantitative', scale=alt.Scale(type='threshold', domain=[1,5,10,50,100,500])),    
        tooltip=['Confirmed', 'Deaths', 'Recovered','Date'],
    ).transform_filter(
        alt.FieldEqualPredicate(field='Date', equal=date)
    ).properties(title=date, width=200, height=150)
    charts_death.append(m) 
    
    r = alt.Chart(gdf_join_slice).mark_geoshape(stroke='gray').encode(  
        fill=alt.Fill('Recovered', type='quantitative', scale=alt.Scale(type='threshold', domain=[1,5,10,50,100,500])),      
        opacity=alt.Opacity('Recovered', type='quantitative', scale=alt.Scale(type='threshold', domain=[1,5,10,50,100,500])),    
        tooltip=['Confirmed', 'Deaths', 'Recovered', 'Date'],
    ).transform_filter(
        alt.FieldEqualPredicate(field='Date', equal=date)
    ).properties(title=date, width=200, height=150)
    charts_recv.append(r)     

v_conf = alt.vconcat(*charts_conf)
v_death = alt.vconcat(*charts_death)
v_recv = alt.vconcat(*charts_recv)

h_conc = alt.hconcat(v_conf, v_death, v_recv
).resolve_scale(
    fill='independent',
    opacity='independent'
)

In [36]:
import hkvportal

In [37]:
dp = hkvportal.Service('https://tsws.hkvservices.nl/mangrove.ws/', '236ebLIeCDdnKbfGe956IvZetIv1')

Dataservice is recognized. https://tsws.hkvservices.nl/mangrove.ws/entry.asmx?WSDL will be used as portal


In [38]:
#dp.create_database(database='2019-nCoV')

In [52]:
dp.update_entry(
    database='2019-nCoV',
    key='22Jan_31Jan',
    data=h_conc.to_html(),
    description='small multiples china'
)

entry available at:
https://tsws.hkvservices.nl/mangrove.ws/data.ashx?function=dataportal.db.getdata&parameters={database:'2019-nCoV',key:'22Jan_31Jan'}&contentType=SET_BY_USER
